# 動物標本を用いた局所冷却実験の解析

## 解析するデータ
データは20170924にnumpyデータが入っている
- 2005-04-16-1/S0025
- 2005-08-24-2/S0011
- 2006-02-18-1/S0011

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
import cv2
import copy
%matplotlib inline

In [3]:
from opmap.RawCam import RawCam
from opmap.VmemMap import VmemMap
from opmap.PhaseMap import PhaseMap
from opmap.PhaseVarianceMap import PhaseVarianceMap

## 2005-04-16-1を解析

In [20]:
window_sizes = [9,10,11,12]
directories = ['2005-04-16-1/C001S0025@', '2005-08-24-2/C001S0011@', '2006-02-18-1/C001S0011@']

In [ ]:
for directory in directories:
    cam = RawCam(path='/mnt/recordings/ExperimentData/%s' % directory,
                 cam_type='sa4',
                 image_width=256, image_height=256,
                 frame_start=0, frame_end=-1)
    cam.setIntROI(val_min=100)

    for window_size in window_sizes:
        result_parent = '/mnt/recordings/AnalysisResult/Seno/opmap/HRS2018/20170929/window%s/%s' % (window_size, directory)
        if not os.path.exists(result_parent):
            os.makedirs(result_parent)

        vmem = VmemMap(cam)
        vmem.smooth(size=window_size)
        vmem.saveImage(os.path.join(result_parent, 'vmem'))
        np.save(os.path.join(result_parent, 'vmem.npy'), vmem.data)
        cmd = 'ffmpeg -r 30 -y -i "{0}/%06d.png" -c:v libx264 -pix_fmt yuv420p -qscale 0 "{0}/movie.mp4"'.format(os.path.join(result_parent, 'vmem'))
        subprocess.call(cmd, shell=True)

        pmap = PhaseMap(vmem, shrink=2)
        pmap.saveImage(os.path.join(result_parent, 'pmap'))
        np.save(os.path.join(result_parent, 'pmap.npy'), pmap.data)
        cmd = 'ffmpeg -r 30 -y -i "{0}/%06d.png" -c:v libx264 -pix_fmt yuv420p -qscale 0 "{0}/movie.mp4"'.format(os.path.join(result_parent, 'pmap'))
        subprocess.call(cmd, shell=True)

        pvmap = PhaseVarianceMap(pmap, size=3)
        pvmap.saveImage(os.path.join(result_parent, 'pvmap'))
        np.save(os.path.join(result_parent, 'pvmap.npy'), pvmap.data)
        cmd = 'ffmpeg -r 30 -y -i "{0}/%06d.png" -c:v libx264 -pix_fmt yuv420p -qscale 0 "{0}/movie.mp4"'.format(os.path.join(result_parent, 'pvmap'))
        subprocess.call(cmd, shell=True)